In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 1. 数据加载和预处理
def load_and_preprocess_data(file_path):
    """
    加载并标准化数据
    """
    # 加载数据
    data = pd.read_csv(file_path)  # 假设数据文件是CSV格式
    X = data.values  # 获取数据的numpy数组

    # 标准化数据
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # 将数据标准化
    return X_scaled

# 2. 自编码器模型设计
def build_autoencoder(input_dim):
    """
    构建一个简单的自编码器模型
    """
    # 输入层
    input_layer = layers.Input(shape=(input_dim,))
    # 编码器部分
    encoded = layers.Dense(8, activation='relu')(input_layer)  # 压缩到8维
    encoded = layers.Dense(5, activation='relu')(encoded)  # 压缩到5维（潜在空间）
    # 解码器部分
    decoded = layers.Dense(8, activation='relu')(encoded)  # 解码回8维
    decoded = layers.Dense(input_dim, activation='linear')(decoded)  # 解码回11维
    # 构建自编码器模型
    autoencoder = models.Model(input_layer, decoded)
    # 编码器模型，用于提取压缩特征
    encoder = models.Model(input_layer, encoded)
    # 编译自编码器模型
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# 3. 训练自编码器
def train_autoencoder(autoencoder, X_scaled, epochs=50, batch_size=256):
    """
    训练自编码器
    """
    autoencoder.fit(X_scaled, X_scaled, epochs=epochs, batch_size=batch_size, shuffle=True)

# 4. 提取压缩特征
def extract_compressed_features(encoder, X_scaled):
    """
    使用训练好的编码器提取压缩特征
    """
    encoded_data = encoder.predict(X_scaled)
    return encoded_data

# 5. 可视化压缩特征
def visualize_compressed_features(encoded_data):
    """
    使用t-SNE可视化压缩特征
    """
    tsne = TSNE(n_components=2)
    tsne_results = tsne.fit_transform(encoded_data)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1])
    plt.title('t-SNE Visualization of Compressed Features')
    plt.show()

# 6. 评估重建效果
def evaluate_reconstruction(autoencoder, X_scaled):
    """
    计算重建误差
    """
    reconstructed = autoencoder.predict(X_scaled)
    reconstruction_error = ((X_scaled - reconstructed) ** 2).mean()
    print(f'Reconstruction Error: {reconstruction_error}')

# 主程序
if __name__ == "__main__":
    # 1. 加载并预处理数据
    file_path = '/home/nlsde/RLmodel/Version2/src/w4_res_fine_state.csv'  # 修改为您的数据文件路径
    X_scaled = load_and_preprocess_data(file_path)
    
    # 2. 构建自编码器模型
    input_dim = X_scaled.shape[1]  # 输入数据的维度（11维）
    autoencoder, encoder = build_autoencoder(input_dim)
    
    # 3. 训练自编码器
    train_autoencoder(autoencoder, X_scaled, epochs=50, batch_size=256)
    
    # 4. 提取压缩特征
    encoded_data = extract_compressed_features(encoder, X_scaled)
    print(f"压缩特征的形状: {encoded_data.shape}")
    
    # 5. 可视化压缩特征
    visualize_compressed_features(encoded_data)
    
    # 6. 评估重建效果
    evaluate_reconstruction(autoencoder, X_scaled)


ModuleNotFoundError: No module named 'tensorflow.python'